# Video Processing for BT Lab
This notebook to test risky code fragments
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [1]:
#import libraries
import os
import cv2
import json
import re
import datetime
import time
import numpy as np

In [2]:
"""delete unwanted files"""
# desired_path = r"D:\Christian\DI_centre_structured\DI_THERMAL_FLIR\3\2 meters\H\WOB\frames"
# os.chdir(desired_path)
# copied_files = [bmp_files for bmp_files in os.listdir() if "Copy" in bmp_files]
# copied_files

'delete unwanted files'

In [3]:
# for copy_file in copied_files:
#     os.remove(copy_file)

In [8]:
def get_video_duration(video_path):
    # Open the video file
    video_capture = cv2.VideoCapture(video_path)

    # Get the total number of frames in the video
    total_frames = video_capture.get(cv2.CAP_PROP_FRAME_COUNT)

    # Get the frames per second (fps) of the video
    fps = video_capture.get(cv2.CAP_PROP_FPS)

    # Calculate the duration in seconds
    duration_seconds = round(total_frames / fps)

    # Convert the duration to a human-readable format (HH:MM:SS)
    video_time = datetime.timedelta(seconds=duration_seconds)

    # Close the video capture object
    video_capture.release()

    return f"duration in seconds: {duration_seconds}, video time: {video_time}"


In [9]:
video_path = "D:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Hold Breath/Arun2.mp4"
duration = get_video_duration(video_path)
print(duration)

duration in seconds: 73, video time: 0:01:13
